In [1]:
#loading time periods
import pyodbc 
from datetime import datetime
from datetime import timezone
from tqdm.notebook import tqdm

server = 'localhost' 
database = 'Kubernetes' 
username = 'sa' 
password = 'test' 
repo_folder = f"C:\\Users\\f2kazemi\\Desktop\\cs848\\Real_Reviewer Recommender\\{database}\\{database}"
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
cursor.execute(f"""SELECT TOP (1000) [Id]
      ,[FromDateTime]
      ,[ToDateTime]
      ,[FirstCommitSha]
      ,[LastCommitSha]
  FROM [{database}].[dbo].[Periods]""")
periods = cursor.fetchall()
commitPeriods = []
for period in periods:
#     utc_time = datetime.strptime(period[1], "%Y-%m-%d %H:%M:%S")
#     print(utc_time.total_seconds())
#     print(period[1])
#     print( period[1].replace(tzinfo=timezone.utc).timestamp())
    start = period[1].replace(tzinfo=timezone.utc).timestamp()
    end = period[2].replace(tzinfo=timezone.utc).timestamp()
    commitPeriods.append([start,end,period[0]])



In [2]:
unknownPeiod = 999
def getPeriod(timestamp):
    for period in commitPeriods:
        if timestamp>=period[0] and timestamp<period[1]:
            return period[2]
    return unknownPeiod;

In [3]:
periods[periods!=-1]

(2, datetime.datetime(2014, 9, 1, 0, 0), datetime.datetime(2014, 12, 1, 0, 0), '2d1149061f10a5e49602f7206f0ab9d95b8986b2', '1c2cea9c108bd800ac2a7bc0d5a3d3b5eb5d544b')

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv(f'./data/{database}.csv',dtype={'fix':'str','fixes':'str','fileschanged':'str'})  

In [9]:
pd.options.mode.chained_assignment = None  # default='warn'

train_data = data[['ns','nd','nf','entropy','la','ld','lt','ndev','age','nuc'
                   ,'exp','rexp','sexp','author_date_unix_timestamp','author_date','contains_bug','commit_hash']]
# train_data['period']=unknownPeiod
for idx in tqdm(range(len(train_data))):
#     train_data.iloc[idx]['period'] = getPeriod(train_data.iloc[idx]['author_date_unix_timestamp'])
    train_data.loc[idx,'period']=getPeriod(train_data.iloc[idx]['author_date_unix_timestamp'])

In [17]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
import warnings

warnings.filterwarnings('ignore')

In [18]:
# now we start creating differen models based on the period of each commit
firstPeriod = min(periods,key=lambda x:x[0])
lastPeriod = max(periods,key=lambda x:x[0])
print(f'first period:{firstPeriod}\nlast period:{lastPeriod}')

first period:(1, datetime.datetime(2014, 6, 1, 0, 0), datetime.datetime(2014, 9, 1, 0, 0), '2c4b3a562ce34cddc3f8218a2c4d11c7310e6d56', '8cd7387cc0bad590b2b63925e4ca8d6fc17a8f61')
last period:(21, datetime.datetime(2019, 6, 1, 0, 0), datetime.datetime(2019, 9, 1, 0, 0), '834db16c9b1a2cf2971abf47ef9cafb41efb89f1', '28f13f720cd0da15ba05a163ee456b301e2ffe48')


In [20]:
# Creating models for different periods
# clean dataset
# Step 1: Omit correlated data
# https://www.analyticsvidhya.com/blog/2020/03/what-is-multicollinearity/
# VIF exceeding 5 or 10 indicates high multicollinearity between this independent variable and the others

from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
import matplotlib.pyplot as plt
import heapq


showDiagram = False
threshold = 0.6

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

periodModels = {}
for periodId in tqdm(range(firstPeriod[0],lastPeriod[0])):
    prior_data = train_data[train_data['period']<periodId]
    # get data for period P and prior to that
    x=train_data[train_data['period'] <= periodId]
    x=x[['ns','nd','nf','entropy','la','ld','lt','ndev','age','nuc','exp','rexp','sexp']]
    y=train_data[train_data['period'] <= periodId]
    y=y['contains_bug']
#     print(f'------------------------------------\nPeriod:{periodId}')
    #illustrate the correlation
    if showDiagram:
        sns.set(rc={'figure.figsize':(12,8)})
        sns.heatmap(x_train.corr().abs(),annot=True)
        plt.show()
    # Omit correlation
    correlations = x.corr()
    omitionIdxs=[]
    for corr in correlations:
        idxs = np.argsort(correlations[corr].to_numpy())
        maxCorrelated = correlations[corr].to_numpy()[idxs[-2]]
        i =0
        while maxCorrelated > threshold:
            if corr not in omitionIdxs:
                omitionIdxs.append(correlations.columns[idxs[-2-i]])
            i += 1
            maxCorrelated = correlations[corr].to_numpy()[idxs[-2-i]]
    x_uncorrelated = x[[column for column in x.columns if column not in omitionIdxs]]
    x=x_uncorrelated
#     print(x_uncorrelated.columns)
    # create a model for them
    model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)
    # scale the data
    scaler = StandardScaler()
    mean = x.mean(axis = 0)
    std = x.std(axis = 0)
    F_X = x[0:3]
    
    # transform data
    x = scaler.fit_transform(x)
    # fit the model
    model.fit(x, y)
    # evaluation of the model
    x_eval = train_data[train_data['period'] == periodId + 1]
    x_eval = x_eval[['ns','nd','nf','entropy','la','ld','lt','ndev','age','nuc','exp','rexp','sexp']]
    x_eval = x_eval[[column for column in x_uncorrelated.columns]]
    x_eval = scaler.fit_transform(x_eval)

    y_eval = train_data[train_data['period'] == periodId + 1]
    y_eval = y_eval['contains_bug']
    #reports
    y_pred = model.predict(x_eval)
    y_prob = model.predict_proba(x_eval)
    score_ = model.score(x_eval,y_eval)
    conf_m = confusion_matrix(y_eval, y_pred)
    report = classification_report(y_eval, y_pred)
    balanced_acc = balanced_accuracy_score(y_eval, y_pred)
    # Print Data
#     print('score_:', score_, end='\n\n')
#     print('conf_m:', conf_m, sep='\n', end='\n\n')
#     print('report:', report, sep='\n')
#     print('balanced_acc: ',balanced_acc,"\n")
#------------------------------------------------
#     print(model.coef_,"\n", model.intercept_,"\n")
#     print(mean, std)    
#     print('f',F_X)
#     print('x',x[0:3])
#     print(model.coef_)
#     k = model.coef_ @ x_eval[1].reshape(8,1) + model.intercept_
#     o = model.intercept_
#     for j in range(model.coef_.shape[1]):
#         print(model.coef_[0][j],x_eval[1][j])
#         o +=  model.coef_[0][j] * x_eval[1][j]
#     print('---->',sigmoid(k))
#     print('====>',sigmoid(o))

    coef={}
    mean={}
    scale={}
    for idx in range(model.coef_.shape[1]):
        coef[ x_uncorrelated.columns[idx]]=model.coef_[0][idx]
        mean[ x_uncorrelated.columns[idx]]=scaler.mean_[idx]
        scale[ x_uncorrelated.columns[idx]]=scaler.scale_[idx]
    coef['intercept']=model.intercept_[0]
    periodModels[periodId]={
        'coef':coef,
        'mean':mean,
        'scale':scale,
        'balanced_acc':balanced_acc
    }
    
#     dotted_product = 0
#     sample_idx=1
#     testing = train_data[train_data['period'] == periodId + 1]
#     testing = testing[['ns','nd','nf','entropy','la','ld','lt','ndev','age','nuc','exp','rexp','sexp']]
#     testing = testing[[column for column in x_uncorrelated.columns]]
#     PR = testing.iloc[sample_idx]
#     for column in testing.columns:
#         temp = (PR[column] - periodModels[periodId]['mean'][column])/periodModels[periodId]['scale'][column]
#         dotted_product += periodModels[periodId]['coef'][column] * (temp)
#     dotted_product += periodModels[periodId]['coef']['intercept']
    
from pprint import pprint 
pprint(periodModels)


{1: {'balanced_acc': 0.6167406965437991,
     'coef': {'age': -0.0146097870082912,
              'exp': 0.3489347435091104,
              'intercept': -1.7123204357959783,
              'la': -0.12823136000625524,
              'lt': 0.2149106735656308,
              'nd': -0.041827293084164656,
              'ndev': 0.22285425533673717,
              'ns': 0.946594937737027},
     'mean': {'age': 5.300879491866319,
              'exp': 66.36195445920303,
              'la': 92.28320683111954,
              'lt': 148.16183932401466,
              'nd': 1.5450664136622392,
              'ndev': 6.566413662239089,
              'ns': 0.7936432637571158},
     'scale': {'age': 10.828779973574893,
               'exp': 102.8329203188523,
               'la': 886.1550235546375,
               'lt': 216.67065823480388,
               'nd': 3.558451913972173,
               'ndev': 9.474616595089744,
               'ns': 0.8478219722607874}},
 2: {'balanced_acc': 0.6161964174301108,
     'co

In [18]:
periodModels

{1: {'coef': {'ns': 0.946594937737027,
   'nd': -0.041827293084164656,
   'la': -0.12823136000625524,
   'lt': 0.2149106735656308,
   'ndev': 0.22285425533673717,
   'age': -0.0146097870082912,
   'exp': 0.3489347435091104,
   'intercept': -1.7123204357959783},
  'mean': {'ns': 0.7936432637571158,
   'nd': 1.5450664136622392,
   'la': 92.28320683111954,
   'lt': 148.16183932401466,
   'ndev': 6.566413662239089,
   'age': 5.300879491866319,
   'exp': 66.36195445920303},
  'scale': {'ns': 0.8478219722607874,
   'nd': 3.558451913972173,
   'la': 886.1550235546375,
   'lt': 216.67065823480388,
   'ndev': 9.474616595089744,
   'age': 10.828779973574893,
   'exp': 102.8329203188523},
  'balanced_acc': 0.6167406965437991},
 2: {'coef': {'ns': 1.0135120932161985,
   'nd': 0.07356544590541002,
   'la': -0.07449798092299123,
   'lt': 0.40038996115519093,
   'age': -0.003651784200472283,
   'exp': 0.2751506070902713,
   'intercept': -1.7432836323081973},
  'mean': {'ns': 0.7218543046357616,
   'n

In [21]:
import json
json.dumps(periodModels[1])

'{"coef": {"ns": 0.946594937737027, "nd": -0.041827293084164656, "la": -0.12823136000625524, "lt": 0.2149106735656308, "ndev": 0.22285425533673717, "age": -0.0146097870082912, "exp": 0.3489347435091104, "intercept": -1.7123204357959783}, "mean": {"ns": 0.7936432637571158, "nd": 1.5450664136622392, "la": 92.28320683111954, "lt": 148.16183932401466, "ndev": 6.566413662239089, "age": 5.300879491866319, "exp": 66.36195445920303}, "scale": {"ns": 0.8478219722607874, "nd": 3.558451913972173, "la": 886.1550235546375, "lt": 216.67065823480388, "ndev": 9.474616595089744, "age": 10.828779973574893, "exp": 102.8329203188523}, "balanced_acc": 0.6167406965437991}'

In [20]:
# now we save model into the database
for i in periods:
#     print(f"""UPDATE [{database}].[dbo].[Periods]
#         SET [ModelParam]='{json.dumps(periodModels[i[0]])}'
#         WHERE Id='{i[0]}'
#     """)
    if i[0] in periodModels:
        cursor.execute(f"""UPDATE [{database}].[dbo].[Periods]
            SET [ModelParam]='{json.dumps(periodModels[i[0]])}'
            WHERE Id='{i[0]+1}'
        """)
        cnxn.commit()


In [26]:
# Now we calculate the defect proneness for each PR
import json

# fetch all the PRs
cursor.execute(f"""SELECT [Id],[Number],[PRCommits],[Metrics],[DefectProneness],[CreatedAtDateTime] from [{database}].[dbo].[PullRequests]""")
PRs = cursor.fetchall() 

i=0
defectProbs={}
print('whole:',len(PRs))
while i < len(PRs):
    PR = PRs[i]
#     if PR[4] and PR[4] is not None :
#         print(i,"here!")
#         i += 1
#         continue
    if PR[3] is None :
        defectProneness = 0
    else:
        metrics = json.loads(PR[3])
        timestamp = PR[5].replace(tzinfo=timezone.utc).timestamp()
        period = getPeriod(timestamp)
        if period in periodModels:
            model = periodModels[period]
            # first we have to normalize the data and only pick the important one
            data = {}
            summation = 0
            for metric in model['coef'].keys():
                if metric !='intercept':
#                     print(f'{metric}:{metrics[metric]}')
                    #first convert it
                    transformed = (metrics[metric] - model['mean'][metric])/model['scale'][metric]
                    summation += model['coef'][metric] * transformed
                else:
                    summation += model['coef'][metric]
            defectProneness = sigmoid(summation)
        else:
            defectProneness = 0
    defectProbs[PR[0]] = defectProneness    
    i += 1
# print(defectProbs)

whole: 47232


In [27]:
defectProbs

{16849604: 0.13762461374217796,
 16851290: 0.9703560395664864,
 16851971: 0.13158414022390494,
 16852740: 0.15495705561247952,
 16853712: 0.04206507850532958,
 16854754: 0.13532067724473262,
 16857672: 0.13876645477661342,
 16860318: 0.14104926060490658,
 16860352: 0.13105858946962645,
 16863804: 0.14077842691051792,
 16864074: 0.12229006588826215,
 16867815: 0.1299897335671265,
 16870162: 0.13738258898949934,
 16870587: 0.20368878384183187,
 16871016: 0.14002833120169386,
 16871260: 0.1464736752073779,
 16871541: 0.14195718165283605,
 16872295: 0.8094211029038112,
 16872531: 0.8575101723979183,
 16872568: 0.12160672943066193,
 16887028: 0.13340410742556152,
 16908241: 0.18331917494049663,
 16908733: 0.14016015055720932,
 16915286: 0.34865257194182475,
 16950707: 0.15005077262580457,
 16950711: 0.15090135074006908,
 16951006: 0.1441879816963946,
 16955630: 0.16343405388412452,
 16957385: 0.043667919339684017,
 16960486: 0.12801591294457007,
 16965005: 0.043667919339684017,
 16965167: 0

In [28]:
# Now we save this probability into the database
# now we save model into the database
for PRId in defectProbs.keys():
    Prob = defectProbs[PRId]
    cursor.execute(f"""UPDATE [{database}].[dbo].[PullRequests]
        SET [DefectProneness]='{Prob}'
        WHERE Id='{PRId}'
    """)
    cnxn.commit()


In [7]:
# train_data.mean(axis = 0)

In [ ]:
#first we create priods based on the author_date

In [ ]:
# model = LogisticRegression(solver='liblinear', C=10.0, random_state=0)

In [ ]:
# x=train_data[['ns','nd','nf','entropy','la','ld','lt','ndev','age','nuc','exp','rexp','sexp']]
# y=train_data['contains_bug']

In [ ]:
# threshold_Idx = 20000
# x_train = x[:-threshold_Idx]
# y_train = y[:-threshold_Idx]
# x_eval = x[-threshold_Idx:]
# y_eval = y[-threshold_Idx:]

In [ ]:
# # clean dataset
# # Step 1: Omit correlated data
# # https://www.analyticsvidhya.com/blog/2020/03/what-is-multicollinearity/
# # VIF exceeding 5 or 10 indicates high multicollinearity between this independent variable and the others

# from statsmodels.stats.outliers_influence import variance_inflation_factor

# def calc_vif(X):

#     # Calculating VIF
#     vif = pd.DataFrame()
#     vif["variables"] = X.columns
#     vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

#     return(vif)

# vifs = calc_vif(x_train)

In [ ]:
# import seaborn as sns
# sns.set(rc={'figure.figsize':(12,8)})
# sns.heatmap(x_train.corr().abs(),annot=True)

In [ ]:
#  x_train.corr()

In [ ]:
# import heapq
# correlations = x_train.corr()
# omitionIdxs=[]
# threshold = 0.6
# for corr in correlations:
#     idxs = np.argsort(correlations[corr].to_numpy())
#     maxCorrelated = correlations[corr].to_numpy()[idxs[-2]]
#     i =0
#     while maxCorrelated > threshold:
#         if corr not in omitionIdxs:
#             print(corr,'->',correlations.columns[idxs[-2-i]])
#             omitionIdxs.append(correlations.columns[idxs[-2-i]])
#         i += 1
#         maxCorrelated = correlations[corr].to_numpy()[idxs[-2-i]]
# print(omitionIdxs)
# x_uncorrelated = x[[column for column in x_train.columns if column not in omitionIdxs]]
# # Step 1 finished

In [ ]:
# scaler = StandardScaler()
# # transform data
# x = scaler.fit_transform(x)

In [ ]:
# print(calc_vif(x_uncorrelated))
# x_train_uncore = x[:-threshold_Idx]
# y_train_uncore = y[:-threshold_Idx]
# x_eval_uncore = x[-threshold_Idx:]
# y_eval_uncore = y[-threshold_Idx:]

In [ ]:
# x_train_final = x_train_uncore
# y_train_final = y_train_uncore
# x_eval_final = x_eval_uncore
# y_eval_final = y_eval_uncore

In [ ]:
# model.fit(x_train_final, y_train_final)

In [ ]:

# y_pred = model.predict(x_eval)
# score_ = model.score(x_eval,y_eval)
# conf_m = confusion_matrix(y_eval, y_pred)
# report = classification_report(y_eval, y_pred)


In [ ]:
# print('score_:', score_, end='\n\n')

In [ ]:
# print('conf_m:', conf_m, sep='\n', end='\n\n')

In [ ]:
# print('report:', report, sep='\n')

In [ ]:
# balanced_accuracy_score(y_eval, y_pred)

In [ ]:
# len(y_pred[y_pred==True])